# <p style="background-color: #f5df18; padding: 10px;"> Foundations of Astronomical Data Science | **Coordinate Transformations** </p>



### <strong>Instructor: <span style="color: darkblue;">Dr. Devontae C. Baxter (UCSD)</span></strong>

Estimated completion time: 🕚 95 minutes



<div style="display: flex;">
    <div style="flex: 1; margin-right: 20px;">
        <h2>Questions</h2>
        <ul>
            <li>How do we transform celestial coordinates from one frame to another and save a subset of the results in files?</li>
        </ul>
    </div>
    <div style="flex: 1;">
        <h2>Learning Objectives</h2>
        <ul>
            <li>Use Python string formatting to compose more complex ADQL queries.</li>
            <li>Work with coordinates and other quantities that have units.</li>
            <li>Download the results of a query and store them in a file.</li>
        </ul>
    </div>
</div>


In the previous lesson, we wrote ADQL queries and used them to select and download data from the Gaia server. In this lesson, we will write a query to select stars from a particular region of the sky.

# Outline
---

We'll start with an example that does a "cone search"; that is, it
selects stars that appear in a circular region of the sky.

Then, to select stars in the vicinity of GD-1, we will:

- Use `Quantity` objects to represent measurements with units.

- Use [Astropy](https://www.astropy.org) to convert coordinates from one frame to another.

- Use the ADQL keywords `POLYGON`, `CONTAINS`, and `POINT` to select
  stars that fall within a polygonal region.

- Submit a query and download the results.

- Store the results in a [FITS file](https://fits.gsfc.nasa.gov/fits_primer.html).

# Working with Units
---


The measurements we will work with are physical quantities, which
means that they have two parts, a value and a unit.
For example, the coordinate 30&deg; has value 30 and its units are degrees.

Until recently, most scientific computation was done with values only;
units were left out of the program altogether, [sometimes with
catastrophic
results](https://en.wikipedia.org/wiki/Mars_Climate_Orbiter#Cause_of_failure).



## The Mars Climate Orbiter Incident 
---
<div>
<img src="https://pbs.twimg.com/media/GE72ty3XsAEAKri.jpg" width="800"/>
</div>

Astropy provides tools for including units explicitly in computations,
which makes it possible to detect errors before they cause disasters.

To use Astropy units, we import them like this:

`u` is an object that contains most common units and all SI units.

You can use `dir` to list them, but you should also [read the
documentation](https://docs.astropy.org/en/stable/units/).

To create a quantity, we multiply a value by a unit:

The result is a `Quantity` object.
Jupyter knows how to display `Quantities` like this:

`Quantities` provides a method called `to` that converts to other units.
For example, we can compute the number of arcminutes in `angle`:

If you add quantities, Astropy converts them to compatible units, if possible:

If the units are not compatible, you get an error. For example:

## <p style="background-color: #f5df18; padding: 10px;"> 🛑 Exercise </p>
---

Create a quantity that represents 5
[arcminutes](https://en.wikipedia.org/wiki/Minute_and_second_of_arc)
and assign it to a variable called `radius`.

Then convert it to degrees.

In [15]:
### your answer here ###

# Selecting a Region

---

One of the most common ways to restrict a query is to select stars in
a particular region of the sky.
For example, here is a query from the [Gaia archive
documentation](https://gea.esac.esa.int/archive-help/adql/examples/index.html)
that selects objects in a circular region centered at (88.8, 7.4) with
a search radius of 5 arcmin (0.08333 deg).

```python
cone_query = """SELECT 
TOP 10 
source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, 0.08333333))
"""
```

This query uses three keywords that are specific to ADQL (not SQL):

- `POINT`: a location in [ICRS
  coordinates](https://en.wikipedia.org/wiki/International_Celestial_Reference_System),
  specified in degrees of right ascension and declination.

- `CIRCLE`: a circle where the first two values are the coordinates of
  the center and the third is the radius in degrees.

- `CONTAINS`: a function that returns `1` if a `POINT` is contained in
  a shape and `0` otherwise. Here is the [documentation of
  `CONTAINS`](https://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html#tth_sEc4.2.12).

A query like this is called a cone search because it selects stars in a cone.
Here is how we run it:

In [12]:
cone_query = """SELECT 
TOP 10 
source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, 0.08333333))
"""

## <p style="background-color: #f5df18; padding: 10px;"> 🛑 Exercise </p>
---

When you are debugging queries like this, you can use `TOP` to limit
the size of the results, but then you still don't know how big the
results will be.

An alternative is to use `COUNT`, which asks for the number of rows
that would be selected, but it does not return them.

In the previous query, replace `TOP 10 source_id` with
`COUNT(source_id)` and run the query again.  How many stars has Gaia
identified in the cone we searched?

In [16]:
### your answer here ####

# Getting GD-1 Data

---

From the Price-Whelan and Bonaca paper, we will try to reproduce [Figure 1](https://www.astroexplorer.org/details/apjlaad7b5f1/eyJrZXlXb3JkcyI6IlByaWNlLVdoZWxhbiIsImF1dGhvciI6IlByaWNlLVdoZWxhbiIsImZyb21ZZWFyIjoyMDE4LCJ0b1llYXIiOjIwMTgsInBhZ2UiOjEsInNob3ciOiIyMDAifQ), which includes this representation of stars likely to belong to GD-1:



<div>
<img src="https://datacarpentry.org/astronomy-python/fig/gd1-4.png" width="1300"/>
</div>


The axes of this figure are defined so the x-axis is aligned with the stars in GD-1, and the y-axis is perpendicular.

* Along the x-axis ($\phi_{1}$) the figure extends from -100 to 20 degrees.

* Along the y-axis ($\phi_{2}$) the figure extends from about -8 to 4 degrees.

Ideally, we would select all stars from this rectangle, but there are more than 10 million of them. This would be difficult to work with, and as anonymous Gaia users, we are limited to 3 million rows in a single query. While we are developing and testing code, it will be faster to work with a smaller dataset.

So we will start by selecting stars in a smaller rectangle near the center of GD-1, from -55 to -45 degrees $\phi_{1}$ and -8 to 4 degrees $\phi_{2}$. First we will learn how to represent these coordinates with Astropy.

# Transforming coordinates

---

Astronomy makes use of many different [coordinate systems](https://en.wikipedia.org/wiki/Celestial_coordinate_system). Transforming between coordinate systems is a common task in observational astronomy, and thankfully, Astropy has abstracted the required spherical trigonometry for us. Below we show the steps to go from Equatorial coordinates (sky coordinates) to Galactic coordinates and finally to a reference frame defined to more easily study GD-1.

Astropy provides a `SkyCoord` object that represents sky coordinates
relative to a specified reference frame.

Let's create a `SkyCoord` object that represents the
approximate coordinates of
[Betelgeuse](https://simbad.u-strasbg.fr/simbad/sim-basic?Ident=Betelgeuse)
(alf Ori) in the ICRS frame.

Note: [ICRS](https://www.iers.org/IERS/EN/Science/ICRS/ICRS.html) is the
"International Celestial Reference System", adopted in 1997 by the
International Astronomical Union.

From the [Simbad](https://simbad.u-strasbg.fr/simbad/sim-basic?Ident=Betelgeuse) database, Betelgeuse is located at `RA=05h55m10.31s, DEC=07d24m25.4s`

Before defining the SkyCoord object, let's first convert these units to degrees. This can be achieved using the `Angle` class from astropy. 

`SkyCoord` objects require units in order to understand the context. There are a number of ways to define
`SkyCoord` objects, in our example, we explicitly specified the coordinates and units and provided a
reference frame.

`SkyCoord` provides the `transform_to` function to transform from one reference frame to another reference frame.
For example, we can transform `coords_icrs` to Galactic coordinates like this:

## 🔔 **COORDINATE VARIABLES**
---

Notice that in the Galactic frame, the coordinates are the variables we usually use for Galactic
longitude and latitude called `l` and `b`, respectively, not `ra` and `dec`. Most reference frames have
different ways to specify coordinates and the `SkyCoord` object will use these names.


To transform to and from GD-1 coordinates, we will use a frame defined
by [Gala](https://gala-astro.readthedocs.io/en/latest/), which is an
Astropy-affiliated library that provides tools for galactic dynamics.

Gala provides
[`GD1Koposov10`](https://gala-astro.readthedocs.io/en/latest/_modules/gala/coordinates/gd1.html),
which is "a Heliocentric spherical coordinate system defined by the
orbit of the GD-1 stream". In this coordinate system, one axis is defined along
the direction of the steam (the x-axis in Figure 1) and one axis is defined
perpendicular to the direction of the stream (the y-axis in Figure 1).
These are called the φ<sub>1</sub> and φ<sub>2</sub> coordinates, respectively.


In [ ]:
pip install gala

In [76]:
from gala.coordinates import GD1Koposov10

We can use it to find the coordinates of Betelgeuse in the GD-1 frame, like this:

## <p style="background-color: #f5df18; padding: 10px;"> 🛑 Exercise </p>
---


Find the location of GD-1 in ICRS coordinates.

1. Create a `SkyCoord` object at 0°, 0° in the GD-1 frame.

2. Transform it to the ICRS frame.

Hint: Because ICRS is a standard frame, it is built into Astropy. You can specify it by name,
`icrs` (as we did with `galactic`).

In [17]:
### your answer here ####

You will notice that the origin of the GD-1 frame maps to `ra=200`, exactly, in ICRS. That is by design.



#  Selecting a rectangle

---

Now that we know how to define coordinate transformations, we are going to use them to get a list of stars that are in GD-1. As we mentioned before, this is a lot of stars, so we are going to start by defining a rectangle that encompasses a small part of GD-1. This is easiest to define in GD-1 coordinates.

As mentioned earlier, let's start by selecting stars in a smaller rectangle near the center of GD-1, from -55 to -45 degrees $\phi_{1}$ and -8 to 4 degrees $\phi_{2}$. First we will learn how to represent these coordinates with Astropy.

In [19]:
### Define the boundaries of the rectangle in $\phi_{1}$ and $\phi_{2}$.

phi1_min =
phi1_max = 
phi2_min = 
phi2_max = 

Throughout this lesson we are going to be defining a rectangle often. Rather than copy and paste multiple lines of code, we will write a function to build the rectangle for us. By having the code contained in a single location, we can easily fix bugs or update our implementation as needed. By choosing an explicit function name our code is also self documenting, meaning its easy for us to understand that we are building a rectangle when we call this function.

To create a rectangle, we will use the following function, which takes the lower and upper bounds as parameters and returns a list of x and y coordinates of the corners of a rectangle starting with the lower left corner and working clockwise.

```python
def make_rectangle(x1, x2, y1, y2):
    """Return the corners of a rectangle."""
    xs = [x1, x1, x2, x2, x1]
    ys = [y1, y2, y2, y1, y1]
    return xs, ys
```

The return value is a tuple containing a list of coordinates in $\phi_{1}$ followed by a list of coordinates in $\phi_{2}$.


In [110]:
### define make_rectangle ### 

def make_rectangle(x1, x2, y1, y2):
    """Return the corners of a rectangle."""
    xs = [x1, x1, x2, x2, x1]
    ys = [y1, y2, y2, y1, y1]
    return xs, ys

Store the coordinates of the corners of
a rectangle in the GD-1 frame as `phi1_rect` and `phi2_rect`.

While it is easier to visualize the regions we want to define in the GD-1 frame, the coordinates in the Gaia catalog are in the ICRS frame. In order to use the rectangle we defined, we need to convert the coordinates from the GD-1 frame to the ICRS frame. We will do this using the SkyCoord object. Fortunately SkyCoord objects can take lists of coordinates, in addition to single values.

Now we can use `transform_to` to convert to ICRS coordinates.

Notice that a rectangle in one coordinate system is not necessarily a rectangle in another. In this example, the result is a (non-rectangular) polygon. This is why we defined our rectangle in the GD-1 frame.

# ♦️ Defining a polygon

---

In order to use this polygon as part of an ADQL query, we have to convert it to a string with a comma-separated list of coordinates, as in this example:

```python
"""POLYGON(143.65, 20.98,
        134.46, 26.39,
        140.58, 34.85,
        150.16, 29.01)
"""
```

`SkyCoord` provides `to_string`, which produces a list of strings.


We can use the Python string function `join` to join `corners_list_str` into a single
string (with spaces between the pairs):

This is almost what we need, but we have to replace the spaces with commas.



This is something we will need to do multiple times. We will write a function to do it for us so we don’t have to copy and paste every time. The following function combines these steps.

```python
def skycoord_to_string(skycoord):
    """Convert a one-dimenstional list of SkyCoord to string for Gaia's query format."""
    corners_list_str = skycoord.to_string()
    corners_single_str = ' '.join(corners_list_str)
    return corners_single_str.replace(' ', ', ')
```

Here is how we use this function:



# 🏗️ Assembling the query

---

Now we are ready to assemble our query to get all of the stars in
the Gaia catalog that are in the small rectangle we defined and
are likely to be part of GD-1 with the criteria we previously defined.

We need `columns` again (as we saw in the previous episode).

And here is the query base we used in the previous lesson:

```python
query3_base = """SELECT
TOP 10
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2
"""
```


Now we will add a `WHERE` clause to select stars in the polygon we defined and start using more descriptive variables for our queries.

In [21]:
### Create query base with additional condition to select stars in the polygon ###

polygon_top10query_base = 

The query base contains format specifiers for `columns` and `sky_point_list`.

We will use `format` to fill in these values.

In [24]:
### Use format to fill in columns and list of sky points ###

polygon_top10query = 

As always, we should take a minute to proof-read the query before we launch it.



In [25]:
### launch and print query ###


Retrieve and display are the results.



## <p style="background-color: #f5df18; padding: 10px;"> 🛑 Exercise </p>
---

Let's remove `TOP 10` and run the query again. The result is bigger than our previous queries, so it will take a little longer. Once done, find the number of stars in the output table.

Hint: To complete this exercise you must take the following steps:

1. Remove `TOP 10` from the query base. 
2. Format the query.
3. Proof-read the query and launch it.
4. Retrieve results.
5. Print the length of the resultant Astropy table. 

In [26]:
### Your answer here ### 

There should be more than 100,000 stars in this polygon, but that’s a manageable size to work with.



# 💾 Saving results

---

This is the set of stars we will work with in the next step.  Since
we have a substantial dataset now, this is a good time to save it.

Storing the data in a file means we can shut down our notebook and
pick up where we left off without running the previous query again.

Astropy `Table` objects provide `write`, which writes the table to disk.

Because the filename ends with `fits`, the table is written in the
[FITS format](https://en.wikipedia.org/wiki/FITS), which preserves the
metadata associated with the table.

If the file already exists, the `overwrite` argument causes it to be
overwritten.

We can use `getsize` to confirm that the file exists and check the size:


In [27]:
from os.path import getsize

MB = 1024 * 1024



### Downloading using Google Colab
---

If you are using Google Colab, it is possible that the data **will not** be saved in the same directory as this notebook.

Use the following code to download the data to your local drive. 


```python
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current working directory:", current_directory)
```

```python
# List the contents of the current directory
contents = os.listdir(current_directory)
print("Contents of the current directory:", contents)

## your file should be saved in the directory '/content/'

from google.colab import files

# Specify the file path of the item you want to download
file_path = '/content/example.fits' # Replace 'example.fits' with the name of your file
  

# Download the file
files.download(file_path)
```

# Summary

---

In this notebook, we composed more complex queries to select stars
within a polygonal region of the sky.  Then we downloaded the results
and saved them in a FITS file.

In the next lesson, we'll reload the data from this file and
replicate the next step in the analysis, using proper motion to
identify stars likely to be in GD-1.

# <p style="background-color: #f5df18; padding: 10px;"> 🗝️ Key points</p>

---

- For measurements with units, use `Quantity` objects that represent units explicitly and check for errors.
- Use the `format` function to compose queries; it is often faster and less error-prone.
- Develop queries incrementally: start with something simple, test it, and add a little bit at a time.
- Once you have a query working, save the data in a local file.  If you shut down the notebook and come back to it later, you can reload the file; you don't have to run the query again.